# Service Areas for Social Facilities

In [1]:
%load_ext watermark
%watermark -a 'eli knaap' -v -d -u -p geopandas,geosnap
%load_ext autoreload
%autoreload 2

Author: eli knaap

Last updated: 2024-03-02

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.18.1

geopandas: 0.14.2
geosnap  : 0.12.1.dev9+g3a1cb0f6de61.d20240110



**Note this notebook requires osmnx**

One way of thinking about isochrones is considering them as service areas. That is, given some travel budget (in time, distance, transit fare, etc), the isochrone represents the service area accessible within that budget.

In [4]:
from geosnap import DataStore
from geosnap.analyze import isochrones_from_gdf
from geosnap.io import get_acs

In [5]:
import geopandas as gpd
import pandana as pdna

In [6]:
datasets = DataStore()

In [7]:
la_tracts = get_acs(datasets, county_fips="06037", years=[2018])

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/io/constructors.py:215: UserWarning: Currency columns unavailable at this resolution; not adjusting for inflation
  warn(


In [9]:
from geosnap.analyze.network import pdna_network_from_gdf

## Walking Service Areas

Imagine we were interested in the service areas around social facilities in Los Angeles

In [10]:
pdna_network_from_gdf?

Signature:
pdna_network_from_gdf(
    gdf,
    network_type='walk',
    twoway=False,
    add_travel_times=False,
    default_speeds=None,
)
Docstring:
Create a pandana.Network object from a geodataframe (via OSMnx graph).

Parameters
----------
gdf : geopandas.GeoDataFrame
    _description_
network_type : str, {"all_private", "all", "bike", "drive", "drive_service", "walk"}
    the type of network to collect from OSM (passed to `osmnx.graph_from_polygon`)
    by default "walk"
twoway : bool, optional
    Whether to treat the pandana.Network as directed or undirected. For a directed network,
    use `twoway=False` (which is the default). For an undirected network (e.g. a
    walk network) where travel can flow in both directions, the network is more
    efficient when twoway=True. This has implications for drive networks or multimodal
    networks where impedance is different depending on travel direction.
add_travel_times : bool, default=False
    whether to use posted travel times fr

In [11]:
la_net = pdna_network_from_gdf(la_tracts)

Generating contraction hierarchies with 10 threads.
Setting CH node vector of size 598263
Setting CH edge vector of size 1697458
Range graph removed 1654378 edges of 3394916
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [12]:
import osmnx as ox

In [13]:
facilities = ox.features.features_from_polygon(la_tracts.unary_union, {'amenity':'social_facility'})

In [14]:
facilities = facilities[ facilities.geometry.type=='Point']

In [15]:
facilities.explore()

In [16]:
from geosnap.analyze import isochrones_from_gdf

In [17]:
isochrones_from_gdf?

Signature:
isochrones_from_gdf(
    origins,
    threshold,
    network,
    network_crs=4326,
    reindex=True,
    algorithm='alpha',
    ratio=0.2,
    allow_holes=False,
)
Docstring:
Create travel isochrones for several origins simultaneously

Parameters
----------
origins : geopandas.GeoDataFrame
    a geodataframe containing the locations of origin point features
threshold: float
    maximum travel distance to define the isochrone, measured in the same
    units as edges_df in the pandana.Network object. If the network was
    created with pandana this is usually meters; if it was created with
    urbanaccess this is usually travel time in minutes.
network : pandana.Network
    pandana Network instance for calculating the shortest path isochrone
    for each origin feature
network_crs : str, int, pyproj.CRS (optional)
    the coordinate system used to store x and y coordinates in the passed
    pandana network. If the network was created with pandana or urbanaccess
    this is ne

### Comparing Hull algorithms

To create the service area, we need to bound the set of reachable intersections using some kind of polygon. The resolution of the service area is dependent on the resolution of the network (i.e. since geosnap and pandana do not interpolate along the road network, greater intersection density will result in a more well-defined polygon).

There are different bounding-polygon algorithms to choose from. The default is the [alpha_shape_auto](https://pysal.org/libpysal/generated/libpysal.cg.alpha_shape_auto.html) algorithm from libpysal, with shapely's [`concave_hull`](https://shapely.readthedocs.io/en/stable/reference/shapely.concave_hull.html) implementation is available as an alternative

In [18]:
alpha = isochrones_from_gdf(facilities, network=la_net, threshold=2000)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:207: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)
/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)


The alpha shape version of the concave hull is the most resource intensive because it tries to optimize the alpha parameter. This also makes it the slowest.

In [19]:
alpha.explore()

In [20]:
ch01 = isochrones_from_gdf(facilities, network=la_net, threshold=2000, algorithm='hull', ratio=0.1)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:207: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)
/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)


The concave hull algorithm in shapely does not do automatic optimization, but requires setting a `ratio` parameter, with smaller values resulting in tighter bounding polygons

In [21]:
ch01.explore()

In [22]:
ch02 = isochrones_from_gdf(facilities, network=la_net, threshold=2000, algorithm='hull', ratio=0.2)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:207: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)
/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)


In [23]:
ch02.explore()

## Driving Service Areas

Be *very* careful with driving times... There are lots of edges with no speed information (requiring us to make strong assumptions), and even at best, these represent free-flow conditions.

To get an automobile network, change  `network_type='drive'` and  `add_travel_times=True`

In [25]:
drive_net = pdna_network_from_gdf(la_tracts, network_type='drive', add_travel_times=True)

Generating contraction hierarchies with 10 threads.
Setting CH node vector of size 174775
Setting CH edge vector of size 461696
Range graph removed 424316 edges of 923392
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


When using travel-time based impedance, travel times are [measured **in seconds**](https://osmnx.readthedocs.io/en/stable/internals-reference.html#osmnx-speed-module)

In [26]:
# 10 minute drive-shed

drive_chrone = isochrones_from_gdf(facilities, network=drive_net, threshold=600, algorithm='hull', ratio=0.08)

/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:207: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)
/Users/knaaptime/Dropbox/projects/geosnap/geosnap/analyze/network.py:57: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  node_ids = network.get_node_ids(origins.centroid.x, origins.centroid.y).astype(int)


In [27]:
# look at only the first record

m=drive_chrone.iloc[[1]].explore()
facilities.iloc[[1]].explore(m=m)

In [28]:
drive_net.edges_df

,from,to,travel_time
0,653656,1718677597,14.2
1,653656,123189012,9.4
2,653656,122697159,9.5
3,653656,1718756337,24.2
4,653681,26427612,8.7
...,...,...,...
461691,11658196513,11658196512,1.0
461692,11658196513,8633426683,7.5
461693,11658196513,8633449287,14.1
461694,11658220519,10282655806,1.7


Remember also this is directed travel. These isochrones represent the area reachable *from* each social service provider; they do not necesssarily represent the origins who can read the provider within a 10 minute drive. That is, drive networks are directed (and usually asymmetric).